# Data Exploration using Spark Dataframes and SQL

## Setting up Spark and other dependencies

In [1]:
pwd

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/Colab\ Notebooks/Datasets

/content/drive/MyDrive/Colab Notebooks/Datasets


In [4]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Datasets'

In [88]:
#!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop2.7.tgz
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/drive/MyDrive/Colab Notebooks/Datasets/spark-3.0.1-bin-hadoop2.7"
#import findspark
#findspark.init()

#import pyspark

In [7]:
!pip install -q findspark

In [8]:
!pip install -q pyspark

In [9]:
import os
import sys

In [12]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [13]:
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession.builder.appName('Spark RDD').getOrCreate()

In [15]:
spark

## Reading File

In [16]:
import pandas as pd
import numpy as np


In [17]:
sc = spark.sparkContext

In [18]:
fileRDD = sc.textFile('/content/drive/MyDrive/Colab Notebooks/Datasets/matches_test.csv')

## Setting up schema

In [19]:
print(f'Number of records in file: {fileRDD.count()}')

Number of records in file: 748


In [20]:
#displaying the first 5 lines
print(fileRDD.take(100))

['Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match', 'IPL-2017,Hyderabad,05/04/17,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh', 'IPL-2017,Pune,06/04/17,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith', 'IPL-2017,Rajkot,07/04/17,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn', 'IPL-2017,Indore,08/04/17,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell', 'IPL-2017,Bangalore,08/04/17,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav', 'IPL-2017,Hyderabad,09/04/17,Gujarat Lions,Sunrisers Hyderabad,Sunrisers Hyderabad,field,normal,0,Sunrisers Hyderabad,0,9,Rashid Khan', 'IPL-2017,Mum

In [21]:
#Removing the header
header = fileRDD.first()
fileRDD = fileRDD.filter(lambda line: line != header)
print(f'Count after removal of header: {fileRDD.count()}')
print('Displaying first 5 records after header removal \n')
print(fileRDD.take(5))

Count after removal of header: 747
Displaying first 5 records after header removal 

['IPL-2017,Hyderabad,05/04/17,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh', 'IPL-2017,Pune,06/04/17,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith', 'IPL-2017,Rajkot,07/04/17,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn', 'IPL-2017,Indore,08/04/17,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell', 'IPL-2017,Bangalore,08/04/17,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav']


In [22]:
header

'Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match'

In [23]:
# Split the data into individual columns
fileRDD = fileRDD.map(lambda line: line.split(','))
print('Data after splitting \n')
print(fileRDD.take(5))

Data after splitting 

[['IPL-2017', 'Hyderabad', '05/04/17', 'Sunrisers Hyderabad', 'Royal Challengers Bangalore', 'Royal Challengers Bangalore', 'field', 'normal', '0', 'Sunrisers Hyderabad', '35', '0', 'Yuvraj Singh'], ['IPL-2017', 'Pune', '06/04/17', 'Mumbai Indians', 'Rising Pune Supergiant', 'Rising Pune Supergiant', 'field', 'normal', '0', 'Rising Pune Supergiant', '0', '7', 'SPD Smith'], ['IPL-2017', 'Rajkot', '07/04/17', 'Gujarat Lions', 'Kolkata Knight Riders', 'Kolkata Knight Riders', 'field', 'normal', '0', 'Kolkata Knight Riders', '0', '10', 'CA Lynn'], ['IPL-2017', 'Indore', '08/04/17', 'Rising Pune Supergiant', 'Kings XI Punjab', 'Kings XI Punjab', 'field', 'normal', '0', 'Kings XI Punjab', '0', '6', 'GJ Maxwell'], ['IPL-2017', 'Bangalore', '08/04/17', 'Royal Challengers Bangalore', 'Delhi Daredevils', 'Royal Challengers Bangalore', 'bat', 'normal', '0', 'Royal Challengers Bangalore', '15', '0', 'KM Jadhav']]


In [24]:
Schema = StructType([
    #StructField("id", StringType(), True)
    StructField("Season", StringType(), True),
    StructField("city", StringType(), True),
    StructField("date", StringType(), True),
    StructField("team1", StringType(), True),
    StructField("team2", StringType(), True),
    StructField("toss_winner", StringType(), True),
    StructField("toss_decision", StringType(), True),
    StructField("result", StringType(), True),
    StructField("dl_applied", StringType(), True),
    StructField("winner", StringType(), True),
    StructField("win_by_runs", StringType(), True),
    StructField("win_by_wickets", StringType(), True),
    StructField("player_of_match", StringType(), True)
])

In [25]:
matches = spark.createDataFrame(data = fileRDD, schema= Schema)

In [26]:
matches.show(5, truncate= False)

+--------+---------+--------+---------------------------+---------------------------+---------------------------+-------------+------+----------+---------------------------+-----------+--------------+---------------+
|Season  |city     |date    |team1                      |team2                      |toss_winner                |toss_decision|result|dl_applied|winner                     |win_by_runs|win_by_wickets|player_of_match|
+--------+---------+--------+---------------------------+---------------------------+---------------------------+-------------+------+----------+---------------------------+-----------+--------------+---------------+
|IPL-2017|Hyderabad|05/04/17|Sunrisers Hyderabad        |Royal Challengers Bangalore|Royal Challengers Bangalore|field        |normal|0         |Sunrisers Hyderabad        |35         |0             |Yuvraj Singh   |
|IPL-2017|Pune     |06/04/17|Mumbai Indians             |Rising Pune Supergiant     |Rising Pune Supergiant     |field        |norma

In [27]:
matches.createOrReplaceTempView('matchesTable')

In [28]:
spark.sql('select * from matchesTable').show()

+--------+---------+--------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+
|  Season|     city|    date|               team1|               team2|         toss_winner|toss_decision|result|dl_applied|              winner|win_by_runs|win_by_wickets|player_of_match|
+--------+---------+--------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+
|IPL-2017|Hyderabad|05/04/17| Sunrisers Hyderabad|Royal Challengers...|Royal Challengers...|        field|normal|         0| Sunrisers Hyderabad|         35|             0|   Yuvraj Singh|
|IPL-2017|     Pune|06/04/17|      Mumbai Indians|Rising Pune Super...|Rising Pune Super...|        field|normal|         0|Rising Pune Super...|          0|             7|      SPD Smith|
|IPL-2017|   Rajkot|07/04/17|       Gujarat Lions|Kolka

## Data Validation

In [29]:
matches.describe().show()

+-------+--------+-------------+--------+-------------------+-------------------+-------------------+-------------+---------+--------------------+-------------------+------------------+------------------+---------------+
|summary|  Season|         city|    date|              team1|              team2|        toss_winner|toss_decision|   result|          dl_applied|             winner|       win_by_runs|    win_by_wickets|player_of_match|
+-------+--------+-------------+--------+-------------------+-------------------+-------------------+-------------+---------+--------------------+-------------------+------------------+------------------+---------------+
|  count|     747|          747|     747|                747|                747|                747|          747|      747|                 747|                747|               747|               747|            747|
|   mean|    null|         null|    null|               null|               null|               null|         null| 

In [30]:
spark.sql('select distinct(team1) from matchesTable').show(truncate = False)

+---------------------------+
|team1                      |
+---------------------------+
|Sunrisers Hyderabad        |
|Chennai Super Kings        |
|Rising Pune Supergiant     |
|Deccan Chargers            |
|Kochi Tuskers Kerala       |
|Rajasthan Royals           |
|Gujarat Lions              |
|Royal Challengers Bangalore|
|Kolkata Knight Riders      |
|Kings XI Punjab            |
|Pune Warriors              |
|Delhi Daredevils           |
|Mumbai Indians             |
|Rising Pune Supergiants    |
|Delhi Capitals             |
+---------------------------+



In [31]:
spark.sql('select distinct(team2) from matchesTable').show(truncate = False)

+---------------------------+
|team2                      |
+---------------------------+
|Sunrisers Hyderabad        |
|Chennai Super Kings        |
|Rising Pune Supergiant     |
|Deccan Chargers            |
|Kochi Tuskers Kerala       |
|Rajasthan Royals           |
|Gujarat Lions              |
|Royal Challengers Bangalore|
|Kolkata Knight Riders      |
|Kings XI Punjab            |
|Pune Warriors              |
|Delhi Daredevils           |
|Mumbai Indians             |
|Rising Pune Supergiants    |
|Delhi Capitals             |
+---------------------------+



In [32]:
spark.sql('select distinct(toss_winner) from matchesTable').show(truncate = False)

+---------------------------+
|toss_winner                |
+---------------------------+
|Sunrisers Hyderabad        |
|Chennai Super Kings        |
|Rising Pune Supergiant     |
|Deccan Chargers            |
|Kochi Tuskers Kerala       |
|Rajasthan Royals           |
|Gujarat Lions              |
|Royal Challengers Bangalore|
|Kolkata Knight Riders      |
|Kings XI Punjab            |
|Pune Warriors              |
|Delhi Daredevils           |
|Mumbai Indians             |
|Rising Pune Supergiants    |
|Delhi Capitals             |
+---------------------------+



There are two records for Pune Team:

1.   Rising Pune Supergiant
2.   Rising Pune Supergiants




In [33]:
matches = matches.na.replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [34]:
matches.select('toss_winner').distinct().show(truncate = False)

+---------------------------+
|toss_winner                |
+---------------------------+
|Sunrisers Hyderabad        |
|Chennai Super Kings        |
|Rising Pune Supergiant     |
|Deccan Chargers            |
|Kochi Tuskers Kerala       |
|Rajasthan Royals           |
|Gujarat Lions              |
|Royal Challengers Bangalore|
|Kolkata Knight Riders      |
|Kings XI Punjab            |
|Pune Warriors              |
|Delhi Daredevils           |
|Mumbai Indians             |
|Delhi Capitals             |
+---------------------------+



In [35]:
matches.select('team1').distinct().show(truncate = False)

+---------------------------+
|team1                      |
+---------------------------+
|Sunrisers Hyderabad        |
|Chennai Super Kings        |
|Rising Pune Supergiant     |
|Deccan Chargers            |
|Kochi Tuskers Kerala       |
|Rajasthan Royals           |
|Gujarat Lions              |
|Royal Challengers Bangalore|
|Kolkata Knight Riders      |
|Kings XI Punjab            |
|Pune Warriors              |
|Delhi Daredevils           |
|Mumbai Indians             |
|Delhi Capitals             |
+---------------------------+



We created only one records for Pune i.e. Rising Pune Supergiant

In [36]:
#replacing the previous table wit hupdated values in matches
matches.createOrReplaceTempView('matchesTable')

In [37]:
spark.sql('select distinct(team1) from matchesTable').show(truncate = False)

+---------------------------+
|team1                      |
+---------------------------+
|Sunrisers Hyderabad        |
|Chennai Super Kings        |
|Rising Pune Supergiant     |
|Deccan Chargers            |
|Kochi Tuskers Kerala       |
|Rajasthan Royals           |
|Gujarat Lions              |
|Royal Challengers Bangalore|
|Kolkata Knight Riders      |
|Kings XI Punjab            |
|Pune Warriors              |
|Delhi Daredevils           |
|Mumbai Indians             |
|Delhi Capitals             |
+---------------------------+



## Data Exploration

In [38]:
matches.describe().show()

+-------+--------+-------------+--------+-------------------+-------------------+-------------------+-------------+---------+--------------------+-------------------+------------------+------------------+---------------+
|summary|  Season|         city|    date|              team1|              team2|        toss_winner|toss_decision|   result|          dl_applied|             winner|       win_by_runs|    win_by_wickets|player_of_match|
+-------+--------+-------------+--------+-------------------+-------------------+-------------------+-------------+---------+--------------------+-------------------+------------------+------------------+---------------+
|  count|     747|          747|     747|                747|                747|                747|          747|      747|                 747|                747|               747|               747|            747|
|   mean|    null|         null|    null|               null|               null|               null|         null| 

### Univariate Analysis



1. Seasons



In [39]:
spark.sql('select distinct(Season) from matchesTable order by Season').show()

+--------+
|  Season|
+--------+
|IPL-2008|
|IPL-2009|
|IPL-2010|
|IPL-2011|
|IPL-2012|
|IPL-2013|
|IPL-2014|
|IPL-2015|
|IPL-2016|
|IPL-2017|
|IPL-2018|
|IPL-2019|
+--------+



We have the matches record from 2008 - 2019

In [40]:
matches.select('Season').distinct().sort('Season').show()

+--------+
|  Season|
+--------+
|IPL-2008|
|IPL-2009|
|IPL-2010|
|IPL-2011|
|IPL-2012|
|IPL-2013|
|IPL-2014|
|IPL-2015|
|IPL-2016|
|IPL-2017|
|IPL-2018|
|IPL-2019|
+--------+



In [41]:
#Number of matches each season
spark.sql('select Season,count(*) from matchesTable group by Season order by Season').show()

+--------+--------+
|  Season|count(1)|
+--------+--------+
|IPL-2008|      58|
|IPL-2009|      57|
|IPL-2010|      60|
|IPL-2011|      73|
|IPL-2012|      74|
|IPL-2013|      76|
|IPL-2014|      53|
|IPL-2015|      58|
|IPL-2016|      60|
|IPL-2017|      59|
|IPL-2018|      60|
|IPL-2019|      59|
+--------+--------+



Season 2011 - 2013 seems to have a bit high number of matches. We'll Explore on this part

In [42]:
matches.groupBy('Season').agg(count('Season').alias('Count')).sort('Season').show()

+--------+-----+
|  Season|Count|
+--------+-----+
|IPL-2008|   58|
|IPL-2009|   57|
|IPL-2010|   60|
|IPL-2011|   73|
|IPL-2012|   74|
|IPL-2013|   76|
|IPL-2014|   53|
|IPL-2015|   58|
|IPL-2016|   60|
|IPL-2017|   59|
|IPL-2018|   60|
|IPL-2019|   59|
+--------+-----+




2. City



In [43]:
spark.sql('select distinct(city) from matchesTable').show()

+------------+
|        city|
+------------+
|   Bangalore|
|       Kochi|
|     Chennai|
|   Centurion|
|      Mumbai|
|   Ahmedabad|
|      Durban|
|     Kolkata|
|   Cape Town|
|  Dharamsala|
|        Pune|
|Johannesburg|
|   Kimberley|
|       Delhi|
|  Chandigarh|
|      Nagpur|
|Bloemfontein|
|      Kanpur|
|   Hyderabad|
|      Rajkot|
+------------+
only showing top 20 rows



In [44]:
spark.sql('select count(distinct(city)) from matchesTable').show()

+--------------------+
|count(DISTINCT city)|
+--------------------+
|                  32|
+--------------------+



There are 32 cities in total where the matches were being played from 2008 - 2019

In [45]:
matches.select('city').distinct().show()

+------------+
|        city|
+------------+
|   Bangalore|
|       Kochi|
|     Chennai|
|   Centurion|
|      Mumbai|
|   Ahmedabad|
|      Durban|
|     Kolkata|
|   Cape Town|
|  Dharamsala|
|        Pune|
|Johannesburg|
|   Kimberley|
|       Delhi|
|  Chandigarh|
|      Nagpur|
|Bloemfontein|
|      Kanpur|
|   Hyderabad|
|      Rajkot|
+------------+
only showing top 20 rows



In [46]:
matches.select('city').distinct().count()

32

In [47]:
#Count of matches in each city
spark.sql('select city,count(*) as Counts from matchesTable group by city order by Counts desc').show(32)

+--------------+------+
|          city|Counts|
+--------------+------+
|        Mumbai|   101|
|       Kolkata|    77|
|         Delhi|    74|
|     Bangalore|    65|
|     Hyderabad|    64|
|       Chennai|    57|
|        Jaipur|    47|
|    Chandigarh|    46|
|          Pune|    38|
|        Durban|    15|
|     Bengaluru|    13|
| Visakhapatnam|    13|
|     Centurion|    12|
|     Ahmedabad|    12|
|        Rajkot|    10|
|        Mohali|    10|
|        Indore|     9|
|    Dharamsala|     9|
|  Johannesburg|     8|
|Port Elizabeth|     7|
|       Cuttack|     7|
|        Ranchi|     7|
|     Cape Town|     7|
|     Abu Dhabi|     7|
|        Raipur|     6|
|       Sharjah|     6|
|         Kochi|     5|
|        Kanpur|     4|
|     Kimberley|     3|
|        Nagpur|     3|
|   East London|     3|
|  Bloemfontein|     2|
+--------------+------+



Mumbai has hosted the highest number of matches

In [48]:
matches.groupBy('city').agg(count('city').alias('Counts')).orderBy(desc('Counts')).show(32)

+--------------+------+
|          city|Counts|
+--------------+------+
|        Mumbai|   101|
|       Kolkata|    77|
|         Delhi|    74|
|     Bangalore|    65|
|     Hyderabad|    64|
|       Chennai|    57|
|        Jaipur|    47|
|    Chandigarh|    46|
|          Pune|    38|
|        Durban|    15|
|     Bengaluru|    13|
| Visakhapatnam|    13|
|     Centurion|    12|
|     Ahmedabad|    12|
|        Rajkot|    10|
|        Mohali|    10|
|        Indore|     9|
|    Dharamsala|     9|
|  Johannesburg|     8|
|Port Elizabeth|     7|
|       Cuttack|     7|
|        Ranchi|     7|
|     Cape Town|     7|
|     Abu Dhabi|     7|
|        Raipur|     6|
|       Sharjah|     6|
|         Kochi|     5|
|        Kanpur|     4|
|     Kimberley|     3|
|        Nagpur|     3|
|   East London|     3|
|  Bloemfontein|     2|
+--------------+------+



3.Toss Winner

In [49]:
spark.sql('select distinct(toss_winner) from matchesTable').show(truncate = False)

+---------------------------+
|toss_winner                |
+---------------------------+
|Sunrisers Hyderabad        |
|Chennai Super Kings        |
|Rising Pune Supergiant     |
|Deccan Chargers            |
|Kochi Tuskers Kerala       |
|Rajasthan Royals           |
|Gujarat Lions              |
|Royal Challengers Bangalore|
|Kolkata Knight Riders      |
|Kings XI Punjab            |
|Pune Warriors              |
|Delhi Daredevils           |
|Mumbai Indians             |
|Delhi Capitals             |
+---------------------------+



These are basically all the teams which participated iin IPL upto 2019

In [50]:
matches.select('toss_winner').distinct().show(truncate = False)

+---------------------------+
|toss_winner                |
+---------------------------+
|Sunrisers Hyderabad        |
|Chennai Super Kings        |
|Rising Pune Supergiant     |
|Deccan Chargers            |
|Kochi Tuskers Kerala       |
|Rajasthan Royals           |
|Gujarat Lions              |
|Royal Challengers Bangalore|
|Kolkata Knight Riders      |
|Kings XI Punjab            |
|Pune Warriors              |
|Delhi Daredevils           |
|Mumbai Indians             |
|Delhi Capitals             |
+---------------------------+



In [51]:
spark.sql('select toss_winner,count(*) as Counts from matchesTable group by toss_winner \
order by Counts desc').show(truncate = False)

+---------------------------+------+
|toss_winner                |Counts|
+---------------------------+------+
|Mumbai Indians             |96    |
|Kolkata Knight Riders      |91    |
|Chennai Super Kings        |89    |
|Kings XI Punjab            |80    |
|Royal Challengers Bangalore|80    |
|Delhi Daredevils           |80    |
|Rajasthan Royals           |77    |
|Sunrisers Hyderabad        |45    |
|Deccan Chargers            |43    |
|Pune Warriors              |20    |
|Gujarat Lions              |15    |
|Rising Pune Supergiant     |13    |
|Delhi Capitals             |10    |
|Kochi Tuskers Kerala       |8     |
+---------------------------+------+



Mumbai Indians have won the highest toss counts

In [52]:
matches.groupBy('toss_winner').agg(count('toss_winner').alias('counts_toss_won'))\
.orderBy(desc('counts_toss_won')).show(truncate = False)

+---------------------------+---------------+
|toss_winner                |counts_toss_won|
+---------------------------+---------------+
|Mumbai Indians             |96             |
|Kolkata Knight Riders      |91             |
|Chennai Super Kings        |89             |
|Kings XI Punjab            |80             |
|Royal Challengers Bangalore|80             |
|Delhi Daredevils           |80             |
|Rajasthan Royals           |77             |
|Sunrisers Hyderabad        |45             |
|Deccan Chargers            |43             |
|Pune Warriors              |20             |
|Gujarat Lions              |15             |
|Rising Pune Supergiant     |13             |
|Delhi Capitals             |10             |
|Kochi Tuskers Kerala       |8              |
+---------------------------+---------------+





3. Toss Decision  



In [53]:
spark.sql('select toss_decision,count(*) as Count from matchesTable group by toss_decision').show()

+-------------+-----+
|toss_decision|Count|
+-------------+-----+
|        field|  457|
|          bat|  290|
+-------------+-----+



In [54]:
from ast import expr
matches.groupBy('toss_decision').agg(count('toss_decision'),(count('toss_decision')/matches.count()).\
                                     alias('percent')).show()

+-------------+--------------------+-------------------+
|toss_decision|count(toss_decision)|            percent|
+-------------+--------------------+-------------------+
|        field|                 457| 0.6117804551539491|
|          bat|                 290|0.38821954484605087|
+-------------+--------------------+-------------------+



After winning the toss 61% of times the team choose to field first

4. Result

In [55]:
spark.sql('select result,count(*) as counts from matchesTable group by result').show()

+---------+------+
|   result|counts|
+---------+------+
|   normal|   736|
|      tie|     9|
|no result|     2|
+---------+------+



In [56]:
matches.groupBy('result').agg(count('result').alias('counts')).show()

+---------+------+
|   result|counts|
+---------+------+
|   normal|   736|
|      tie|     9|
|no result|     2|
+---------+------+



5. DL applied

In [57]:
spark.sql('select dl_applied,count(*),count(*)/747 as percent from matchesTable group by dl_applied').show(3)

+----------+--------+--------------------+
|dl_applied|count(1)|             percent|
+----------+--------+--------------------+
|         0|     728|  0.9745649263721553|
|         1|      19|0.025435073627844713|
+----------+--------+--------------------+



Duck Worth luis was used in 19 games total

In [58]:
matches.groupBy('dl_applied').agg(count('dl_applied'), (count('dl_applied')/matches.count())\
                                  .alias('percent')).show()

+----------+-----------------+--------------------+
|dl_applied|count(dl_applied)|             percent|
+----------+-----------------+--------------------+
|         0|              728|  0.9745649263721553|
|         1|               19|0.025435073627844713|
+----------+-----------------+--------------------+



6. Winner

In [59]:
spark.sql('select winner,count(*) as counts from matchesTable group by winner order by counts desc')\
.show(truncate = False)

+---------------------------+------+
|winner                     |counts|
+---------------------------+------+
|Mumbai Indians             |109   |
|Chennai Super Kings        |98    |
|Kolkata Knight Riders      |92    |
|Royal Challengers Bangalore|83    |
|Kings XI Punjab            |81    |
|Rajasthan Royals           |75    |
|Delhi Daredevils           |66    |
|Sunrisers Hyderabad        |56    |
|Deccan Chargers            |29    |
|Rising Pune Supergiant     |15    |
|Gujarat Lions              |13    |
|Pune Warriors              |12    |
|Delhi Capitals             |10    |
|Kochi Tuskers Kerala       |6     |
|N.A                        |2     |
+---------------------------+------+



In [60]:
matches.groupBy('winner').agg(count('winner').alias('matches_won')).orderBy(desc('matches_won'))\
.show(truncate = False)

+---------------------------+-----------+
|winner                     |matches_won|
+---------------------------+-----------+
|Mumbai Indians             |109        |
|Chennai Super Kings        |98         |
|Kolkata Knight Riders      |92         |
|Royal Challengers Bangalore|83         |
|Kings XI Punjab            |81         |
|Rajasthan Royals           |75         |
|Delhi Daredevils           |66         |
|Sunrisers Hyderabad        |56         |
|Deccan Chargers            |29         |
|Rising Pune Supergiant     |15         |
|Gujarat Lions              |13         |
|Pune Warriors              |12         |
|Delhi Capitals             |10         |
|Kochi Tuskers Kerala       |6          |
|N.A                        |2          |
+---------------------------+-----------+



Mumbai Indians have won the maximum number of matches followd by Chennai Super Kings

N.A are the matches which were having no results

7. Win By Runs

In [61]:
spark.sql('select max(win_by_runs) from matchesTable').show()

+----------------+
|max(win_by_runs)|
+----------------+
|              98|
+----------------+



In [62]:
matches.select(max('win_by_runs')).show()

+----------------+
|max(win_by_runs)|
+----------------+
|              98|
+----------------+



8. Win By Wickets

In [63]:
spark.sql('select max(win_by_wickets) from matchesTable').show()

+-------------------+
|max(win_by_wickets)|
+-------------------+
|                  9|
+-------------------+



There is no win by 10 wickets


In [64]:
matches.select(max('win_by_wickets')).show()

+-------------------+
|max(win_by_wickets)|
+-------------------+
|                  9|
+-------------------+



In [65]:
spark.sql('select player_of_match, count(*) as counts from matchesTable group by player_of_match order by counts desc').show(truncate = False)

+---------------+------+
|player_of_match|counts|
+---------------+------+
|CH Gayle       |21    |
|AB de Villiers |20    |
|RG Sharma      |17    |
|DA Warner      |17    |
|MS Dhoni       |17    |
|YK Pathan      |16    |
|SR Watson      |15    |
|SK Raina       |14    |
|G Gambhir      |13    |
|MEK Hussey     |12    |
|V Kohli        |12    |
|AM Rahane      |12    |
|V Sehwag       |11    |
|A Mishra       |11    |
|DR Smith       |11    |
|AD Russell     |11    |
|KA Pollard     |10    |
|JH Kallis      |10    |
|SP Narine      |9     |
|SE Marsh       |9     |
+---------------+------+
only showing top 20 rows



In [66]:
matches.groupBy('player_of_match').agg((count('player_of_match')).alias('POM_count')).orderBy(desc('POM_count')).show()

+---------------+---------+
|player_of_match|POM_count|
+---------------+---------+
|       CH Gayle|       21|
| AB de Villiers|       20|
|      RG Sharma|       17|
|      DA Warner|       17|
|       MS Dhoni|       17|
|      YK Pathan|       16|
|      SR Watson|       15|
|       SK Raina|       14|
|      G Gambhir|       13|
|     MEK Hussey|       12|
|        V Kohli|       12|
|      AM Rahane|       12|
|       V Sehwag|       11|
|       A Mishra|       11|
|       DR Smith|       11|
|     AD Russell|       11|
|     KA Pollard|       10|
|      JH Kallis|       10|
|      SP Narine|        9|
|       SE Marsh|        9|
+---------------+---------+
only showing top 20 rows



### Multivariate Analysis

In [67]:
spark.sql('select count(*), count(*)/747 as percent from matchesTable where toss_winner = winner').show()

+--------+------------------+
|count(1)|           percent|
+--------+------------------+
|     390|0.5220883534136547|
+--------+------------------+



The teams which have won the toss have won the matches about 50% of times

This means that winning the toss has no significant impact on the outcome of the match

In [68]:
field_df = spark.sql('select * from matchesTable where toss_decision = "field"')

In [69]:
field_df.groupBy('city').agg(count('city').alias('counts')).orderBy(desc('counts')).show(32)

+-------------+------+
|         city|counts|
+-------------+------+
|       Mumbai|    66|
|    Bangalore|    56|
|      Kolkata|    49|
|        Delhi|    42|
|    Hyderabad|    36|
|   Chandigarh|    30|
|       Jaipur|    28|
|      Chennai|    21|
|         Pune|    21|
|    Bengaluru|    13|
|       Mohali|     9|
|   Dharamsala|     8|
|       Indore|     8|
|Visakhapatnam|     8|
|       Rajkot|     7|
|    Ahmedabad|     6|
| Johannesburg|     6|
|    Centurion|     6|
|       Durban|     5|
|      Cuttack|     5|
|       Ranchi|     4|
|      Sharjah|     4|
|       Kanpur|     4|
|       Raipur|     3|
|    Abu Dhabi|     3|
|    Cape Town|     3|
|        Kochi|     3|
|    Kimberley|     1|
|       Nagpur|     1|
| Bloemfontein|     1|
+-------------+------+



In [70]:
bat_df = spark.sql('select * from matchesTable where toss_decision = "bat"')

In [71]:
bat_df.groupBy('city').agg(count('city').alias('counts')).orderBy(desc('counts')).show(32)

+--------------+------+
|          city|counts|
+--------------+------+
|       Chennai|    36|
|        Mumbai|    35|
|         Delhi|    32|
|     Hyderabad|    28|
|       Kolkata|    28|
|        Jaipur|    19|
|          Pune|    17|
|    Chandigarh|    16|
|        Durban|    10|
|     Bangalore|     9|
|Port Elizabeth|     7|
|     Centurion|     6|
|     Ahmedabad|     6|
| Visakhapatnam|     5|
|     Cape Town|     4|
|     Abu Dhabi|     4|
|        Ranchi|     3|
|        Raipur|     3|
|        Rajkot|     3|
|   East London|     3|
|  Johannesburg|     2|
|     Kimberley|     2|
|        Nagpur|     2|
|       Cuttack|     2|
|         Kochi|     2|
|       Sharjah|     2|
|    Dharamsala|     1|
|  Bloemfontein|     1|
|        Indore|     1|
|        Mohali|     1|
+--------------+------+



We can see the toss preferances as per each city stadiums

Some cities are preferred for fielding first while some for batting

In [72]:
team1 = matches.select('team1')

In [73]:
team2 = matches.select('team2')

In [74]:
teams = team1.union(team2)

In [75]:
teams.count()

1494

In [76]:
teams.show(truncate = False)

+---------------------------+
|team1                      |
+---------------------------+
|Sunrisers Hyderabad        |
|Mumbai Indians             |
|Gujarat Lions              |
|Rising Pune Supergiant     |
|Royal Challengers Bangalore|
|Gujarat Lions              |
|Kolkata Knight Riders      |
|Royal Challengers Bangalore|
|Delhi Daredevils           |
|Sunrisers Hyderabad        |
|Kings XI Punjab            |
|Royal Challengers Bangalore|
|Rising Pune Supergiant     |
|Kolkata Knight Riders      |
|Delhi Daredevils           |
|Gujarat Lions              |
|Rising Pune Supergiant     |
|Delhi Daredevils           |
|Sunrisers Hyderabad        |
|Royal Challengers Bangalore|
+---------------------------+
only showing top 20 rows



In [77]:
teams = teams.withColumnRenamed('team1','teams')

In [78]:
teams.show(truncate = False)

+---------------------------+
|teams                      |
+---------------------------+
|Sunrisers Hyderabad        |
|Mumbai Indians             |
|Gujarat Lions              |
|Rising Pune Supergiant     |
|Royal Challengers Bangalore|
|Gujarat Lions              |
|Kolkata Knight Riders      |
|Royal Challengers Bangalore|
|Delhi Daredevils           |
|Sunrisers Hyderabad        |
|Kings XI Punjab            |
|Royal Challengers Bangalore|
|Rising Pune Supergiant     |
|Kolkata Knight Riders      |
|Delhi Daredevils           |
|Gujarat Lions              |
|Rising Pune Supergiant     |
|Delhi Daredevils           |
|Sunrisers Hyderabad        |
|Royal Challengers Bangalore|
+---------------------------+
only showing top 20 rows



In [79]:
teams.groupBy('teams').agg(count('teams').alias('Matches_played')).orderBy(desc('Matches_played'))\
.show(truncate = False)

+---------------------------+--------------+
|teams                      |Matches_played|
+---------------------------+--------------+
|Mumbai Indians             |184           |
|Kolkata Knight Riders      |177           |
|Royal Challengers Bangalore|176           |
|Kings XI Punjab            |175           |
|Chennai Super Kings        |162           |
|Delhi Daredevils           |159           |
|Rajasthan Royals           |144           |
|Sunrisers Hyderabad        |106           |
|Deccan Chargers            |75            |
|Pune Warriors              |46            |
|Gujarat Lions              |30            |
|Rising Pune Supergiant     |30            |
|Delhi Capitals             |16            |
|Kochi Tuskers Kerala       |14            |
+---------------------------+--------------+



Mumbai Indians have played the highest number of matches.

In [80]:
team_matches = teams.groupBy('teams').agg(count('teams').alias('Matches_played')).\
orderBy(desc('Matches_played'))
toss = matches.groupBy('toss_winner').agg(count('toss_winner').alias('toss_wins'))

In [81]:
team_toss = team_matches.join(toss, toss['toss_winner'] == team_matches['teams'], 'inner')
team_toss = team_toss.drop('toss_winner')

In [82]:
team_toss = team_toss.withColumn('percent', team_toss.toss_wins / team_toss.Matches_played).\
orderBy(desc('percent'))

In [83]:
team_toss.show(truncate = False)

+---------------------------+--------------+---------+-------------------+
|teams                      |Matches_played|toss_wins|percent            |
+---------------------------+--------------+---------+-------------------+
|Delhi Capitals             |16            |10       |0.625              |
|Deccan Chargers            |75            |43       |0.5733333333333334 |
|Kochi Tuskers Kerala       |14            |8        |0.5714285714285714 |
|Chennai Super Kings        |162           |89       |0.5493827160493827 |
|Rajasthan Royals           |144           |77       |0.5347222222222222 |
|Mumbai Indians             |184           |96       |0.5217391304347826 |
|Kolkata Knight Riders      |177           |91       |0.5141242937853108 |
|Delhi Daredevils           |159           |80       |0.5031446540880503 |
|Gujarat Lions              |30            |15       |0.5                |
|Kings XI Punjab            |175           |80       |0.45714285714285713|
|Royal Challengers Bangal

Delhi Capitals have a slightly higher percentage of toss wins.

In [84]:
winner = matches.groupBy('winner').agg(count('winner').alias('wins')).orderBy(desc('wins'))

In [85]:
team_wins = team_matches.join(winner, team_matches['teams'] == winner['winner'], 'inner')
team_wins = team_wins.drop('winner')

In [86]:
team_wins = team_wins.withColumn('percent', team_wins.wins / team_wins.Matches_played)\
.orderBy(desc('percent'))

In [87]:
team_wins.show(truncate = False)

+---------------------------+--------------+----+-------------------+
|teams                      |Matches_played|wins|percent            |
+---------------------------+--------------+----+-------------------+
|Delhi Capitals             |16            |10  |0.625              |
|Chennai Super Kings        |162           |98  |0.6049382716049383 |
|Mumbai Indians             |184           |109 |0.592391304347826  |
|Sunrisers Hyderabad        |106           |56  |0.5283018867924528 |
|Rajasthan Royals           |144           |75  |0.5208333333333334 |
|Kolkata Knight Riders      |177           |92  |0.519774011299435  |
|Rising Pune Supergiant     |30            |15  |0.5                |
|Royal Challengers Bangalore|176           |83  |0.4715909090909091 |
|Kings XI Punjab            |175           |81  |0.46285714285714286|
|Gujarat Lions              |30            |13  |0.43333333333333335|
|Kochi Tuskers Kerala       |14            |6   |0.42857142857142855|
|Delhi Daredevils   



1.   Delhi Capitals have the highest win percentages. But they have played less number of matches as compared to others
2.   Though Mumbai Indians have highest match win counts but their win percentage is lower and ranked at third.
3.   Delhi Daredevils and Deccan Chargers had low win percentages. But it changed with changes made to the team. Its visible from the win percenatges of Delhi Capitals and Sunrisers Hyderabad respectively.



